In [ ]:
import math
import numpy as np

import time
from numba import jit

from quad_funcs import *

def quadratic_sieve(n, B=None, verbose=False):
    

    if B is None: B = math.ceil(math.exp(math.sqrt(math.log(n) * math.log(math.log(n)))))
    
    if verbose: print("B =", B)
    

    
    smooths = []
    
    S = []
    
    print("finding primes...")
    b_primes = erat_sieve(B)
    K = len(b_primes)
    
    b_prod = reduce(mul, b_primes, 1)
    
    print("finding a_i's....")
    #A1 - find ± a_i where a_i^2 = n (mod p_i)
    a_s = [] 
    for p in tqdm(b_primes[1:]): #what to do for 2?
        a_s.append(find_a(n % p, p))
    
    
    
    
    ## Trying different x values
    
    print("sieving...")
    
    x = round(math.sqrt(n))
    x_initial = x
    
    widgets = ['b-smooth count: ', Percentage(), ' ', Bar(marker='-',left='[',right=']'),
           ' ', ETA(), ' ']

    pbar = ProgressBar(widgets=widgets, maxval=K+1)
    pbar.start()
    print(x)
    
    while len(smooths) <= K:
        #t = [(0, time.time())]
        #print(len(smooths), "/", x - x_initial , end = "\r")
        y = (x**2) % n
        #t.append((1, time.time()))
        
        #print("here")
        if is_bsmooth(b_prod, y):
            #t.append((2, time.time()))
            
            factors = b_factor(b_primes, y)
            #factors = b_factor2(b_primes, b_prod, y) #attempt at faster one
            
            #t.append((3, time.time()))
            
            
            if verbose: print("x =", x, ": ", factors)
                
            S.append((x, x**2 - n))
            
            smooths.append(factors)
            
            #t.append((4, time.time()))
            
        
        #t.append((5, time.time()))
        
        #for r in range(1, len(t)):
        #    print(t[r-1][0], "-", t[r][0], ": ", t[r][1] - t[r-1][1], end=", ")
        #print("")
            
        x += 1
        
        pbar.update(len(smooths)) #this adds a little symbol at each iteration
    pbar.finish()
    
    ## Gaussian elimination ....
    print("mod 2ing matrix")
    
    M = np.array([mod2(s) for s in smooths])
    
    print("reducing matrix")
   
    red_M, elim_rows = matrix_structure_hueristic(M)
    
    print("finding dependence")
    
    reduced_M, dep_rows = reduce_matrix(M)
    
    row_sum = [sum(elim_rows[:i]) for i in range(len(elim_rows))]
    
    rows = [row_sum.index(d) for d in dep_rows]
    print(rows)
    #rows contains the indices of M that sum to zero
    
    print("solving for x,y...")
    
    x = 1
    for r in rows:
        x = (x * S[r][0]) % n
    
    y = 1 #how to use a_i's??
    for j in range(K):
        p = sum([smooths[r][j] for r in rows])/2
        #y = (y * b_primes[j] ** p) % n
        y = bin_ladder(y * b_primes[j], int(p), n)
    
    #y = y % n
    #y = 1
    #for a in a_s:
    #    #y *= bin_ladder(a, 1/2, n)
    #    y = (y * a**1/2) % n
    

    
    d = gcd(x-y, n)
    
    print(n, "=", d, "*", n/d)
    assert n % d == 0
    
    return d
        
        
    
    
quadratic_sieve(16921456439215439701, verbose=False)
#539873, B=19
#16921456439215439701
#46839566299936919234246726809

finding primes...


 40%|███▉      | 14166/35596 [00:00<00:00, 71590.32it/s]

finding a_i's....


100%|██████████| 35596/35596 [00:00<00:00, 67516.80it/s]
b-smooth count:   0% [                                        ] ETA:   0:04:40 

sieving...
4113569793


b-smooth count:  74% [-----------------------------           ] ETA:   0:01:21 

In [ ]:
M = np.array([[1, 0, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 0, 0,],
 [0, 1, 1, 0, 0, 0, 1, 0,],
 [1, 1, 1, 0, 0, 0, 0, 0,],
 [1, 0, 1, 1, 0, 1, 0, 0,],
 [1, 1, 0, 1, 1, 1, 0, 0]])

print(M)

In [ ]:
matrix_structure_hueristic(M, verbose=True)